In [1]:
import pandas as pd
import gzip
import json

In [2]:
categories=['AMAZON_FASHION','Appliances']
categories_dict={'AMAZON_FASHION':'Fashion','Appliances':'Appliances'}
global list_izes
list_sizes=[]

In [3]:
#Functions to unzip and read data from json files
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    #return pd.DataFrame.from_dict(df, orient='index')
    if 'meta' in path:
        list_sizes.append(len(df))
    return df

def getFrame(r_lst,m_lst):
    r_final_list = list(map(lambda x: pd.DataFrame.from_dict(x, orient='index'), r_lst))
    m_final_list = list(map(lambda x: pd.DataFrame.from_dict(x, orient='index'), m_lst))
    r_data=pd.concat(r_final_list)
    m_data=pd.concat(m_final_list)
    return r_data,m_data

In [4]:
def getPath(category):
    r=[]
    m=[]
    for item in category:
        review_path='./Data/'+item+'.json.gz'
        meta_path='./Data/meta_'+item+'.json.gz'
        r.append(review_path)
        m.append(meta_path)
    return r,m

In [5]:
def get_Data(categories):
    all_data=[]
    df_reviews=[]
    df_meta=[]
    for item in categories:
        review_path='./Data/'+item+'.json.gz'
        meta_path='./Data/meta_'+item+'.json.gz'
        df_reviews.append(getDF(review_path))
        df_meta.append(getDF(meta_path))
    #return pd.DataFrame.from_dict(df_reviews, orient='index'),pd.DataFrame.from_dict(df_meta, orient='index')
    f_review,f_meta=getFrame(df_reviews,df_meta)
    return f_review,f_meta

In [6]:
list_sizes=[]
review_data,meta_data=get_Data(categories)

In [7]:
review_data=review_data.reset_index(drop=True)
meta_data=meta_data.reset_index(drop=True)

In [8]:
list_sizes

[186637, 30445]

In [9]:
meta_data.shape

(217082, 18)

In [10]:
#Adding category column to metadata files
meta_data['category']=''*meta_data.shape[0]

In [11]:
meta_data.index

RangeIndex(start=0, stop=217082, step=1)

In [12]:
for k,row in meta_data.iterrows():
    if k<list_sizes[0]:
        row['category']=categories_dict[categories[0]]
    else:
        row['category']=categories_dict[categories[1]]

In [13]:
print(review_data.shape)
print(meta_data.shape)

(1486413, 12)
(217082, 18)


In [14]:
meta_data=meta_data.drop('date',axis=1)

In [15]:
#Cleaning up meta data
index_lst=meta_data.loc[meta_data['title'].str.startswith('var ',na=False)].index
for val in index_lst:
    meta_data.loc[val,'title']=meta_data.loc[val,'category']

In [16]:
review_data.to_csv('./Data/review_data.csv')

In [17]:
meta_data.to_csv('./Data/meta_data.csv')